# StoryLand AI

**Transform books into travel adventures.**

This notebook demonstrates a multi-agent system built with Google's Agent Development Kit that automatically creates personalized travel itineraries from your favorite books. Simply provide a book title, and watch as specialized AI agents work together to research locations, discover landmarks, and compose a complete journey through the story's world.

## Setup

In [27]:
# Import libraries
import os
import sys
import json
import time
import requests
import uuid
import logging
from typing import List, Optional, Dict, Any
from dataclasses import dataclass
from pathlib import Path
from dotenv import load_dotenv

# Import Google ADK components
from google.genai import types
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import FunctionTool, google_search, AgentTool

# Import logging
from common.logging import setup_logger, get_tracer

# Load environment
load_dotenv()

# Enable DEBUG logging to see step-by-step execution
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s'
)

# Setup logging
logger = setup_logger("storyland_demo")
tracer = get_tracer()

print("Environment Setup:")
print(f"   Google API Key: {'OK' if os.getenv('GOOGLE_API_KEY') else 'MISSING'}")
print(f"   Log Level: DEBUG (step-by-step execution visible)")

Environment Setup:
   Google API Key: OK
   Log Level: DEBUG (step-by-step execution visible)


#### Define BookInfo Data Structure

Create a dataclass to hold book information returned from Google Books API.

In [28]:
@dataclass
class BookInfo:
    """Simple book information"""
    title: str
    authors: List[str]
    description: Optional[str] = None
    published_date: Optional[str] = None
    categories: List[str] = None
    image_url: Optional[str] = None  # Book cover image URL

print("Created BookInfo dataclass")

Created BookInfo dataclass


#### Google Books API Search

Function that calls the Google Books API to search for books and parse the results.

In [29]:
def search_books(title: str, author: Optional[str] = None, max_results: int = 5) -> List[BookInfo]:
    """
    Search Google Books API and return matching books.
    
    Args:
        title: Book title to search for
        author: Optional author name
        max_results: Maximum number of results
        
    Returns:
        List of BookInfo objects
    """
    logger.info(f"Searching Google Books for: {title}")
    
    try:
        # Build query
        query_parts = []
        if title:
            query_parts.append(f"intitle:{title}")
        if author:
            query_parts.append(f"inauthor:{author}")
        
        query = "+".join(query_parts)
        
        # API request
        url = "https://www.googleapis.com/books/v1/volumes"
        params = {
            "q": query,
            "maxResults": max_results,
            "printType": "books"
        }
        
        print(f"Searching: {query}")
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        
        data = response.json()
        items = data.get("items", [])
        
        # Parse results
        results = []
        for item in items:
            vol = item.get("volumeInfo", {})
            
            # Extract image URL (prefer larger sizes)
            image_links = vol.get("imageLinks", {})
            image_url = (
                image_links.get("large") or
                image_links.get("medium") or
                image_links.get("thumbnail") or 
                image_links.get("smallThumbnail") or
                None
            )
            
            results.append(BookInfo(
                title=vol.get("title", "Unknown"),
                authors=vol.get("authors", []),
                description=vol.get("description"),
                published_date=vol.get("publishedDate"),
                categories=vol.get("categories", []),
                image_url=image_url
            ))
        
        logger.info(f"Found {len(results)} books")
        print(f"Found {len(results)} books")
        
        # Log all found books
        for i, book in enumerate(results):
            author_str = ", ".join(book.authors) if book.authors else "Unknown"
            print(f"   [{i}] {book.title} by {author_str} ({book.published_date or 'N/A'})")
        
        return results
        
    except Exception as e:
        logger.error(f"Google Books search failed: {str(e)}")
        raise

print("Created search_books() function")

Created search_books() function


#### Create FunctionTool for Book Search

**Simple Search Pattern** - Agent searches and gets the first/best match:

1. **Agent calls tool** with book title (e.g., "nightingale")
2. **Tool searches** Google Books API → Finds matching books
3. **Tool logs** all books found for visibility
4. **Tool returns** the first/best match automatically
5. **Agent continues** with the selected book

This is a streamlined approach where the tool automatically selects the most relevant book.

In [30]:
def search_book(title: str, author: str = "") -> str:
    """
    Search Google Books API and return the first/best matching book.
    
    This tool searches for books and automatically selects the most relevant result.
    It logs all found books for visibility, then returns the first match.
    
    Use this tool when the user provides a book title to get book metadata.
    
    Args:
        title: The book title to search for (e.g., "The Nightingale", "1984").
              Can be partial or complete title.
        author: Optional author name to narrow results (e.g., "Kristin Hannah").
               Leave empty if unknown.
    
    Returns:
        JSON string with book metadata:
        {
            "book_title": "...", 
            "author": "...", 
            "published_date": "...", 
            "description": "...",
            "categories": [...],
            "image_url": "..."
        }
    
    Example:
        User says: "Find information about nightingale"
        Tool searches and finds 5 books
        Tool logs all found books
        Tool returns the first/best match (e.g., "The Nightingale" by Kristin Hannah)
    """
    logger.info(f"search_book called: title='{title}', author='{author}'")
    
    try:
        # Search for books
        books = search_books(title=title, author=author or None, max_results=5)
        
        if not books:
            logger.warning(f"No books found for: {title}")
            return json.dumps({"error": "No books found", "query": {"title": title, "author": author}})
        
        # Select the first/best match
        selected = books[0]
        author_str = ", ".join(selected.authors) if selected.authors else "Unknown"
        
        print(f"Selected: [{0}] {selected.title} by {author_str}")
        logger.info(f"Selected book: {selected.title} by {author_str}")
        
        # Return the selected book's metadata
        result = {
            "book_title": selected.title,
            "author": author_str,
            "published_date": selected.published_date,
            "description": selected.description,
            "categories": selected.categories or [],
            "image_url": selected.image_url
        }
        
        return json.dumps(result, ensure_ascii=False)
        
    except Exception as e:
        logger.error(f"Book search failed: {str(e)}", exc_info=True)
        return json.dumps({"error": str(e), "type": type(e).__name__})

# Create FunctionTool (no confirmation needed)
google_books_tool = FunctionTool(search_book)

print("Created FunctionTool:")
print(f"   Function: search_book")
print(f"   Pattern: Search -> Log -> Auto-select first match")

Created FunctionTool:
   Function: search_book
   Pattern: Search -> Log -> Auto-select first match


## Model Configuration

In [31]:
# Configure Gemini model
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504]
)

model = Gemini(
    model="gemini-2.5-flash-lite",
    api_key=os.getenv("GOOGLE_API_KEY"),
    retry_options=retry_config
)

print(f"Model configured: {model.model}")

Model configured: gemini-2.5-flash-lite


## Agent Orchestration

We'll build a multi-agent workflow in steps:
1. **Create specialized agents** 
2. **Compose agents** using SequentialAgent and ParallelAgent
3. **Set up session** and runner
4. **Execute workflow** and collect results
5. **Display results**

In [32]:
print("Creating Specialized Agents...")
print("   (These agents are generic and can work with any book)")
print()

# Agent 1: Get book metadata using FunctionTool
book_metadata_agent = LlmAgent(
    name="book_metadata_extractor",
    model=model,
    tools=[google_books_tool],
    instruction="""Extract book metadata for the book provided by the user.

Use search_book_metadata to get:
- Title, authors, description
- Categories, ISBN, published date
- Book cover image URL

Return JSON: {"book_title": "title", "author": "author", "description": "desc", "isbn": "isbn", "book_image_url": "url", "published_date": "date", "categories": []}"""
)
print("   - book_metadata_agent (FunctionTool)")

# Agent 2: Get book context using google_search (built-in)
book_context_agent = LlmAgent(
    name="book_context_extractor",
    model=model,
    tools=[google_search],
    instruction="""Research the book to find:
- Primary locations where the story takes place
- Time period/era/setting
- Main themes

Use google_search to find this information about the book.

Return JSON: {"primary_locations": ["city1", "country1"], "time_period": "era", "themes": ["theme1", "theme2"]}"""
)
print("   - book_context_agent (google_search)")

# Agent 3: Find cities
city_agent = LlmAgent(
    name="city_finder",
    model=model,
    tools=[google_search],
    instruction="""Find real cities to visit based on the book information provided.

Use google_search to validate cities are real and visitable.
Return JSON: {"cities": [{"name": "City", "country": "Country", "relevance": "connection to book"}]}"""
)
print("   - city_agent (google_search)")

# Agent 4: Find landmarks
landmark_agent = LlmAgent(
    name="landmark_finder",
    model=model,
    tools=[google_search],
    instruction="""Find landmarks related to the book.

Use google_search to find specific places mentioned in or related to the book.
Return JSON: {"landmarks": [{"name": "Place", "city": "City", "connection": "how it relates"}]}"""
)
print("   - landmark_agent (google_search)")

# Agent 5: Find author sites
author_agent = LlmAgent(
    name="author_site_finder",
    model=model,
    tools=[google_search],
    instruction="""Find author-related sites.

Use google_search to find museums, birthplaces, or other sites related to the author.
Return JSON: {"author_sites": [{"name": "Site", "type": "museum/birthplace", "city": "City"}]}"""
)
print("   - author_agent (google_search)")

# Agent 6: Compose trip (NO tools - just synthesis)
trip_composer_agent = LlmAgent(
    name="trip_composer",
    model=model,
    tools=[],
    instruction="""You are a literary travel itinerary planner.

Synthesize book info, cities, landmarks, and author sites into a complete travel plan.

Return JSON:
{
  "itinerary": {
    "cities": [
      {
        "name": "City",
        "country": "Country",
        "days_suggested": 1-3,
        "overview": "Brief overview",
        "stops": [
          {
            "name": "Place",
            "type": "landmark|museum|cafe|restaurant|bookstore|neighborhood|street|park|viewpoint|cemetery|library|other",
            "reason": "Connection to book",
            "time_of_day": "morning|afternoon|evening|full_day",
            "notes": "Tips"
          }
        ]
      }
    ],
    "summary_text": "Engaging trip overview"
  }
}"""
)
print("   - trip_composer_agent (no tools)")

print(f"\nCreated 6 generic agents (reusable for any book)")

Creating Specialized Agents...
   (These agents are generic and can work with any book)

   - book_metadata_agent (FunctionTool)
   - book_context_agent (google_search)
   - city_agent (google_search)
   - landmark_agent (google_search)
   - author_agent (google_search)
   - trip_composer_agent (no tools)

Created 6 generic agents (reusable for any book)


### Step 2: Compose Workflow with SequentialAgent & ParallelAgent

**Architecture**:
```
SequentialAgent (overall_workflow)
├─ SequentialAgent (book_extraction)
│  ├─ book_metadata_agent
│  └─ book_context_agent
├─ ParallelAgent (parallel_discovery) ⚡ CONCURRENT
│  ├─ city_agent
│  ├─ landmark_agent
│  └─ author_agent
└─ trip_composer_agent
```

#### Step 1: Sequential Book Extraction

Create a `SequentialAgent` that runs book metadata extraction followed by book context research.

In [33]:
print("Creating Workflow Structure...")
print()

# Step 1: Sequential book extraction (metadata + context)
book_extraction_seq = SequentialAgent(
    name="book_extraction",
    sub_agents=[book_metadata_agent, book_context_agent]
)
print("   - SequentialAgent: book_metadata -> book_context")

Creating Workflow Structure...

   - SequentialAgent: book_metadata -> book_context


#### Step 2: Parallel Discovery

Create a `ParallelAgent` that runs city, landmark, and author discovery **concurrently** for better performance.

In [34]:
# Step 2: Parallel discovery (cities + landmarks + author)
parallel_discovery = ParallelAgent(
    name="parallel_discovery",
    sub_agents=[city_agent, landmark_agent, author_agent]
)
print("   - ParallelAgent: city + landmark + author (CONCURRENT)")
print("   * These 3 agents will run at the same time!")

   - ParallelAgent: city + landmark + author (CONCURRENT)
   * These 3 agents will run at the same time!


#### Step 3: Combine into Overall Workflow

Create the top-level `SequentialAgent` that orchestrates:
1. Book extraction (sequential)
2. Parallel discovery (concurrent)
3. Trip composition

In [35]:
# Step 3: Combine into overall workflow
workflow = SequentialAgent(
    name="overall_workflow",
    sub_agents=[
        book_extraction_seq,   # Step 1: Book info
        parallel_discovery,     # Step 2: Discovery
        trip_composer_agent     # Step 3: Compose
    ]
)
print("   - SequentialAgent: extraction -> discovery -> composition")
print(f"\nWorkflow structure created!")
print(f"   Total steps: 3 (sequential)")
print(f"   Parallel tasks: 3 (in step 2)")

   - SequentialAgent: extraction -> discovery -> composition

Workflow structure created!
   Total steps: 3 (sequential)
   Parallel tasks: 3 (in step 2)


### Configure Your Book

Change the book title and author below, then run the cells to create a travel itinerary!

In [36]:
# ============================================================
# CONFIGURE YOUR BOOK HERE
# ============================================================
BOOK_TITLE = "gone with the wind"
AUTHOR = ""  # Optional - leave empty if unknown
# ============================================================

print(f"Book: {BOOK_TITLE}")
print(f"Author: {AUTHOR or 'Unknown'}")
print(f"\nConfiguration set!")

Book: gone with the wind
Author: Unknown

Configuration set!


### Step 3: Set Up Runner and Session

Create a runner and session to execute the workflow.

In [37]:
print("Setting up Runner and Session...")
print()

# Create session service
session_service = InMemorySessionService()
print("   - Created InMemorySessionService")

# Create runner with the workflow
runner = Runner(
    agent=workflow,
    app_name="storyland",
    session_service=session_service
)
print("   - Created Runner with workflow")

# Create a new session
session_id = str(uuid.uuid4())
await session_service.create_session(
    app_name="storyland",
    user_id="user1",
    session_id=session_id,
    state={"book_title": BOOK_TITLE, "author": AUTHOR}
)
print(f"   - Created session: {session_id[:8]}...")

print(f"\nReady to execute!")

2025-11-19 22:41:27,947 - WARNING - google_adk.google.adk.runners - App name mismatch detected. The runner is configured with app name "storyland", but the root agent was loaded from "/Users/osa/Documents/storyland/storyland-ai/.venv/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".


Setting up Runner and Session...

   - Created InMemorySessionService
   - Created Runner with workflow
   - Created session: fa58c5f0...

Ready to execute!


### Step 4: Execute Workflow

Run the orchestrated workflow and collect results from all agents.

### Step 4: Execute Workflow

Run the orchestrated workflow and collect the final result.

In [38]:
import time

print("="*70)
print("EXECUTING WORKFLOW")
print("="*70)
print(f"Book: {BOOK_TITLE}")
print(f"Author: {AUTHOR or 'Unknown'}\n")

orchestration_start = time.time()

# Create the prompt
prompt = f"""Create a literary travel itinerary for "{BOOK_TITLE}" by {AUTHOR}.

Execute all steps and return the complete combined results."""

user_message = types.Content(
    role='user',
    parts=[types.Part(text=prompt)]
)

print(f"USER REQUEST:")
print(f"{prompt}\n")
print("Starting workflow execution...\n")
print("="*70)

# Run workflow and observe events
final_response = None
event_count = 0

async for event in runner.run_async(
    user_id="user1",
    session_id=session_id,
    new_message=user_message
):
    event_count += 1
    author = event.author if event.author else 'system'
    
    print(f"\nEvent #{event_count}: {author}")
    print("-" * 70)
    
    # Show content parts
    if event.content and event.content.parts:
        for i, part in enumerate(event.content.parts):
            # Show function calls (what agent is requesting)
            if hasattr(part, 'function_call') and part.function_call:
                func_call = part.function_call
                print(f"\n  TOOL CALL: {func_call.name}")
                print(f"     ID: {func_call.id if hasattr(func_call, 'id') else 'N/A'}")
                if hasattr(func_call, 'args') and func_call.args:
                    args_dict = dict(func_call.args)
                    print(f"     Arguments:")
                    for key, value in args_dict.items():
                        print(f"       - {key}: {value}")
            
            # Show function responses (what tool returned)
            if hasattr(part, 'function_response') and part.function_response:
                func_resp = part.function_response
                print(f"\n  TOOL RESPONSE: {func_resp.name}")
                if hasattr(func_resp, 'response') and func_resp.response:
                    resp_dict = dict(func_resp.response)
                    print(f"     Full Response:")
                    for key, value in resp_dict.items():
                        # Pretty print the response
                        value_str = str(value)
                        if len(value_str) > 500:
                            # Format JSON nicely if it's JSON
                            try:
                                import json
                                parsed = json.loads(value_str)
                                value_str = json.dumps(parsed, indent=2)
                            except:
                                pass
                        print(f"       {key}:")
                        for line in value_str.split('\n'):
                            print(f"         {line}")
            
            # Show text content (agent's response)
            if hasattr(part, 'text') and part.text:
                print(f"\n  TEXT RESPONSE:")
                print("     " + "-" * 66)
                # Print full text with proper indentation
                for line in part.text.split('\n'):
                    print(f"     {line}")
                print("     " + "-" * 66)
    
    # Mark final responses
    if event.is_final_response():
        print(f"\n  FINAL RESPONSE from {author}")
        final_response = event

print("\n" + "="*70)

# Extract result from final response
result_data = {}
if final_response and final_response.content and final_response.content.parts:
    for part in final_response.content.parts:
        if hasattr(part, 'text') and part.text:
            json_start = part.text.find('{')
            json_end = part.text.rfind('}') + 1
            
            if json_start >= 0 and json_end > json_start:
                try:
                    result_data = json.loads(part.text[json_start:json_end])
                    break
                except json.JSONDecodeError as e:
                    logger.error(f"Failed to parse JSON: {e}")

total_duration = time.time() - orchestration_start

# Summary
print("\n" + "="*70)
print("EXECUTION SUMMARY")
print("="*70)
print(f"Total duration: {total_duration:.1f}s")
print(f"Total events: {event_count}")
print(f"Result keys: {len(result_data)}")
if result_data:
    print(f"\nResult structure:")
    for key in result_data.keys():
        value = result_data[key]
        if isinstance(value, list):
            print(f"   - {key}: list with {len(value)} items")
        elif isinstance(value, dict):
            print(f"   - {key}: dict with {len(value)} keys")
        else:
            print(f"   - {key}: {type(value).__name__}")
print("\nWORKFLOW COMPLETE!")
print("="*70)

2025-11-19 22:41:27,969 - INFO - google_adk.google.adk.models.google_llm - Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-11-19 22:41:27,969 - DEBUG - google_adk.google.adk.models.google_llm - 
LLM Request:
-----------------------------------------------------------
System Instruction:
Extract book metadata for the book provided by the user.

Use search_book_metadata to get:
- Title, authors, description
- Categories, ISBN, published date
- Book cover image URL

Return JSON: {"book_title": "title", "author": "author", "description": "desc", "isbn": "isbn", "book_image_url": "url", "published_date": "date", "categories": []}

You are an agent. Your internal name is "book_metadata_extractor".
-----------------------------------------------------------
Config:
{'tools': [{}]}
-----------------------------------------------------------
Contents:
{"parts":[{"text":"Create a literary travel itinerary for \"gone with the wind\" by .

EXECUTING WORKFLOW
Book: gone with the wind
Author: Unknown

USER REQUEST:
Create a literary travel itinerary for "gone with the wind" by .

Execute all steps and return the complete combined results.

Starting workflow execution...



2025-11-19 22:41:28,467 - DEBUG - httpcore.http11 - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=UTF-8'), (b'Vary', b'Origin'), (b'Vary', b'X-Origin'), (b'Vary', b'Referer'), (b'Content-Encoding', b'gzip'), (b'Date', b'Thu, 20 Nov 2025 03:41:28 GMT'), (b'Server', b'scaffolding on HTTPServer2'), (b'X-XSS-Protection', b'0'), (b'X-Frame-Options', b'SAMEORIGIN'), (b'X-Content-Type-Options', b'nosniff'), (b'Server-Timing', b'gfet4t7; dur=411'), (b'Alt-Svc', b'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'), (b'Transfer-Encoding', b'chunked')])
2025-11-19 22:41:28,469 - INFO - httpx - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"
2025-11-19 22:41:28,469 - DEBUG - httpcore.http11 - receive_response_body.started request=<Request [b'POST']>
2025-11-19 22:41:28,471 - DEBUG - httpcore.http11 - receive_response_body.complete
2025-11-19 22:


Event #1: book_metadata_extractor
----------------------------------------------------------------------

  TOOL CALL: search_book
     ID: adk-7874784a-1238-4729-aedc-a87fd756eb53
     Arguments:
       - title: Gone with the Wind
Searching: intitle:Gone with the Wind


2025-11-19 22:41:28,914 - DEBUG - urllib3.connectionpool - https://www.googleapis.com:443 "GET /books/v1/volumes?q=intitle%3AGone+with+the+Wind&maxResults=5&printType=books HTTP/1.1" 200 None
2025-11-19 22:41:28 - storyland_demo - INFO - Found 5 books
2025-11-19 22:41:28,918 - INFO - storyland_demo - Found 5 books
2025-11-19 22:41:28 - storyland_demo - INFO - Selected book: Gone with the Wind by Margaret Mitchell
2025-11-19 22:41:28,926 - INFO - storyland_demo - Selected book: Gone with the Wind by Margaret Mitchell
2025-11-19 22:41:28,929 - INFO - google_adk.google.adk.models.google_llm - Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-11-19 22:41:28,930 - DEBUG - google_adk.google.adk.models.google_llm - 
LLM Request:
-----------------------------------------------------------
System Instruction:
Extract book metadata for the book provided by the user.

Use search_book_metadata to get:
- Title, authors, description
- Categor

...


Event #6: city_finder
----------------------------------------------------------------------

  TEXT RESPONSE:
     ------------------------------------------------------------------
     The following itinerary offers a journey through the settings and historical context of Margaret Mitchell's "Gone With the Wind," focusing on real places that evoke the era and spirit of the novel.
     
     **A Literary Journey Through Scarlett's South**
     
     **Day 1-3: Atlanta, Georgia - The Rebirth of a City**
     
     *   **Atlanta History Center:** Begin your exploration at the Atlanta History Center in Buckhead. Here, you can delve into the history of the Civil War, explore exhibits on the film "Gone With the Wind," and tour historic houses like the Swan House. The center also houses the Margaret Mitchell House and Museum in Midtown, where you can visit the apartment where Mitchell wrote her iconic novel.
     *   **Cyclorama:** A significant part of the Atlanta History Center is the C

2025-11-19 22:41:42,893 - DEBUG - httpcore.http11 - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=UTF-8'), (b'Vary', b'Origin'), (b'Vary', b'X-Origin'), (b'Vary', b'Referer'), (b'Content-Encoding', b'gzip'), (b'Date', b'Thu, 20 Nov 2025 03:41:42 GMT'), (b'Server', b'scaffolding on HTTPServer2'), (b'X-XSS-Protection', b'0'), (b'X-Frame-Options', b'SAMEORIGIN'), (b'X-Content-Type-Options', b'nosniff'), (b'Server-Timing', b'gfet4t7; dur=7493'), (b'Alt-Svc', b'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'), (b'Transfer-Encoding', b'chunked')])
2025-11-19 22:41:42,894 - INFO - httpx - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"
2025-11-19 22:41:42,894 - DEBUG - httpcore.http11 - receive_response_body.started request=<Request [b'POST']>
2025-11-19 22:41:42,896 - DEBUG - httpcore.http11 - receive_response_body.complete
2025-11-19 22


Event #7: author_site_finder
----------------------------------------------------------------------

  TEXT RESPONSE:
     ------------------------------------------------------------------
     Here is a literary travel itinerary for "Gone with the Wind" by Margaret Mitchell. This itinerary focuses on locations in Georgia and South Carolina that evoke the atmosphere and historical context of the novel.
     
     ## A Journey Through Scarlett's South: A "Gone With the Wind" Literary Itinerary
     
     This itinerary is designed to immerse you in the world of Margaret Mitchell's epic novel, "Gone with the Wind," by exploring historical sites, charming towns, and the landscapes that shaped this enduring story.
     
     **Day 1-3: Atlanta, Georgia – The Heart of the Confederacy and Scarlett's Rebirth**
     
     *   **Day 1: Arrival and Literary Roots**
         *   Arrive in Atlanta and check into your hotel.
         *   Begin your literary journey at the **Margaret Mitchell Hous

2025-11-19 22:41:49,523 - DEBUG - httpcore.http11 - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=UTF-8'), (b'Vary', b'Origin'), (b'Vary', b'X-Origin'), (b'Vary', b'Referer'), (b'Content-Encoding', b'gzip'), (b'Date', b'Thu, 20 Nov 2025 03:41:49 GMT'), (b'Server', b'scaffolding on HTTPServer2'), (b'X-XSS-Protection', b'0'), (b'X-Frame-Options', b'SAMEORIGIN'), (b'X-Content-Type-Options', b'nosniff'), (b'Server-Timing', b'gfet4t7; dur=6600'), (b'Alt-Svc', b'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'), (b'Transfer-Encoding', b'chunked')])
2025-11-19 22:41:49,525 - INFO - httpx - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"
2025-11-19 22:41:49,526 - DEBUG - httpcore.http11 - receive_response_body.started request=<Request [b'POST']>
2025-11-19 22:41:49,528 - DEBUG - httpcore.http11 - receive_response_body.complete
2025-11-19 22


Event #8: trip_composer
----------------------------------------------------------------------

  TEXT RESPONSE:
     ------------------------------------------------------------------
     ```json
     {
       "itinerary": {
         "cities": [
           {
             "name": "Atlanta",
             "country": "USA",
             "days_suggested": 3,
             "overview": "Atlanta, the vibrant capital of Georgia, serves as the starting point for this literary journey. Once a hub of Confederate power and a symbol of Southern resilience, the city's history is deeply intertwined with the narrative of 'Gone With the Wind'.",
             "stops": [
               {
                 "name": "Margaret Mitchell House",
                 "type": "museum",
                 "reason": "This is the actual apartment building where Margaret Mitchell wrote her epic novel, 'Gone With the Wind'. It offers insights into her life and the creation of the book.",
                 "time_of_day": "mo

### Step 5: Display Results

Parse and display the collected results from all agents.

In [39]:
# Extract itinerary from result
itinerary = result_data.get('itinerary', {})

# Display results
print("="*70)
print("FINAL RESULT")
print("="*70)

if not itinerary:
    print("\nNo itinerary data found in result")
    print(f"\nResult data keys: {list(result_data.keys())}")
else:
    # Extract cities from itinerary
    cities_list = itinerary.get('cities', [])
    
    if cities_list:
        print(f"\nTRAVEL ITINERARY - {len(cities_list)} City Plan(s)")
        print("="*70)
        
        for i, city_plan in enumerate(cities_list, 1):
            print(f"\nCity #{i}: {city_plan.get('name')}, {city_plan.get('country')}")
            print(f"   Days suggested: {city_plan.get('days_suggested', 1)}")
            
            if city_plan.get('overview'):
                print(f"\n   Overview:")
                print(f"   {city_plan.get('overview')}")
            
            stops = city_plan.get('stops', [])
            if stops:
                print(f"\n   Stops ({len(stops)}):")
                print("   " + "-"*66)
                
                for j, stop in enumerate(stops, 1):
                    tod = stop.get('time_of_day', 'unknown').upper()
                    stop_type = stop.get('type', 'other')
                    
                    print(f"\n   {j}. {stop.get('name')}")
                    print(f"      Time: {tod}")
                    print(f"      Type: {stop_type}")
                    
                    if stop.get('reason'):
                        print(f"      Connection to book:")
                        print(f"         {stop.get('reason')}")
                    
                    if stop.get('notes'):
                        print(f"      Notes:")
                        print(f"         {stop.get('notes')}")
                
                print("   " + "-"*66)
            
            print()  # Blank line between cities
        
        # Summary
        if itinerary.get('summary_text'):
            print("="*70)
            print("TRIP SUMMARY")
            print("="*70)
            print(f"\n{itinerary.get('summary_text')}")
            print()
    else:
        print("\nNo cities found in itinerary")

# Statistics
print("\n" + "="*70)
print("STATISTICS")
print("="*70)
if itinerary and 'cities' in itinerary:
    total_cities = len(itinerary.get('cities', []))
    total_stops = sum(len(city.get('stops', [])) for city in itinerary.get('cities', []))
    total_days = sum(city.get('days_suggested', 0) for city in itinerary.get('cities', []))
    
    print(f"Total cities: {total_cities}")
    print(f"Total stops: {total_stops}")
    print(f"Total days: {total_days}")
    
    # Count stop types
    stop_types = {}
    for city in itinerary.get('cities', []):
        for stop in city.get('stops', []):
            stop_type = stop.get('type', 'other')
            stop_types[stop_type] = stop_types.get(stop_type, 0) + 1
    
    if stop_types:
        print(f"\nStops by type:")
        for stop_type, count in sorted(stop_types.items(), key=lambda x: x[1], reverse=True):
            print(f"   - {stop_type}: {count}")
else:
    print("No data available")

print("\nDisplay complete!")
print("="*70)

FINAL RESULT

TRAVEL ITINERARY - 5 City Plan(s)

City #1: Atlanta, USA
   Days suggested: 3

   Overview:
   Atlanta, the vibrant capital of Georgia, serves as the starting point for this literary journey. Once a hub of Confederate power and a symbol of Southern resilience, the city's history is deeply intertwined with the narrative of 'Gone With the Wind'.

   Stops (4):
   ------------------------------------------------------------------

   1. Margaret Mitchell House
      Time: MORNING
      Type: museum
      Connection to book:
         This is the actual apartment building where Margaret Mitchell wrote her epic novel, 'Gone With the Wind'. It offers insights into her life and the creation of the book.
      Notes:
         Guided tours are highly recommended to fully appreciate the context of Mitchell's writing.

   2. Atlanta History Center (including the Cyclorama)
      Time: AFTERNOON
      Type: museum
      Connection to book:
         The center houses the "Cyclorama: Th